In [1]:
# Cell 1: Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.postprocessing import ThresholdOptimizer


In [2]:
# Cell 2: Load data and assemble text feature
df = pd.read_csv("job_applicant_dataset.csv")
df['text'] = (
    df['Resume'].fillna('') + ' '
    + df['Job Description'].fillna('') + ' '
    + df['Job Roles'].fillna('')
)


In [3]:
# Cell 3: Define feature matrix and target
cat_cols = ['Gender', 'Race', 'Ethnicity']
num_cols = ['Age']

X = df[['text'] + cat_cols + num_cols]
y = df['Best Match']


In [4]:
# Cell 4: Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [5]:
# Cell 5: Preprocessing + pipeline
text_transformer = TfidfVectorizer(max_features=5000)
cat_transformer  = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
num_transformer  = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  MinMaxScaler(feature_range=(0, 1)))
])

preprocessor = ColumnTransformer([
    ('tfidf',   text_transformer, 'text'),
    ('onehot',  cat_transformer,  cat_cols),
    ('scale',   num_transformer,  num_cols),
])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier',   MultinomialNB(alpha=1.0))
])


In [6]:
# Cell 6: Train baseline model
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf',
                                                  TfidfVectorizer(max_features=5000),
                                                  'text'),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Gender', 'Race',
                                                   'Ethnicity']),
                                                 ('scale',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age'])])),
                ('classifier', MultinomialNB())])

In [7]:
# Cell 7: Fairness-metrics helper (all diffs as absolute values)
def compute_and_print_metrics(
    y_true, y_pred,
    sensitive_feature,
    privileged_value,
    unprivileged_value,
    label="Default"
):
    acc = accuracy_score(y_true, y_pred)
    print(f"{label} Accuracy: {acc:.3f}\n")
    print(f"--- {label} Fairness Metrics for Sensitive Attribute: Gender ---")

    sf = sensitive_feature
    priv_mask   = sf == privileged_value
    unpriv_mask = sf == unprivileged_value

    n_priv   = priv_mask.sum()
    n_unpriv = unpriv_mask.sum()

    tp_priv   = ((y_pred == 1) & (y_true == 1) & priv_mask).sum()
    fp_priv   = ((y_pred == 1) & (y_true == 0) & priv_mask).sum()
    tn_priv   = ((y_pred == 0) & (y_true == 0) & priv_mask).sum()
    fn_priv   = ((y_pred == 0) & (y_true == 1) & priv_mask).sum()
    tp_unpriv = ((y_pred == 1) & (y_true == 1) & unpriv_mask).sum()
    fp_unpriv = ((y_pred == 1) & (y_true == 0) & unpriv_mask).sum()
    tn_unpriv = ((y_pred == 0) & (y_true == 0) & unpriv_mask).sum()
    fn_unpriv = ((y_pred == 0) & (y_true == 1) & unpriv_mask).sum()

    sr_priv   = (tp_priv + fp_priv) / n_priv
    sr_unpriv = (tp_unpriv + fp_unpriv) / n_unpriv

    tpr_priv   = tp_priv   / (tp_priv   + fn_priv)   if (tp_priv   + fn_priv)   > 0 else np.nan
    tpr_unpriv = tp_unpriv / (tp_unpriv + fn_unpriv) if (tp_unpriv + fn_unpriv) > 0 else np.nan

    fpr_priv   = fp_priv   / (fp_priv   + tn_priv)   if (fp_priv   + tn_priv)   > 0 else np.nan
    fpr_unpriv = fp_unpriv / (fp_unpriv + tn_unpriv) if (fp_unpriv + tn_unpriv) > 0 else np.nan

    fnr_priv   = fn_priv   / (tp_priv   + fn_priv)   if (tp_priv   + fn_priv)   > 0 else np.nan
    fnr_unpriv = fn_unpriv / (tp_unpriv + fn_unpriv) if (tp_unpriv + fn_unpriv) > 0 else np.nan

    te_priv   = fp_priv   / fn_priv   if fn_priv   > 0 else np.nan
    te_unpriv = fp_unpriv / fn_unpriv if fn_unpriv > 0 else np.nan

    # Absolute differences
    sp_diff   = abs(sr_unpriv   - sr_priv)
    tpr_diff  = abs(tpr_unpriv  - tpr_priv)
    fpr_diff  = abs(fpr_unpriv  - fpr_priv)
    oaed      = 0.5 * ((fpr_unpriv - fpr_priv) + (fnr_unpriv - fnr_priv))
    oaed_diff = abs(oaed)
    te_diff   = abs(te_unpriv   - te_priv)

    print(f"Statistical Parity Difference (|unpriv - priv|): {sp_diff:.3f}")
    print(f"Equalized Odds TPR Difference (abs):             {tpr_diff:.3f}")
    print(f"Equalized Odds FPR Difference (abs):             {fpr_diff:.3f}")
    print(f"Average Odds Difference (OAED, abs):             {oaed_diff:.3f}")
    print(f"Treatment Equality Difference (TED, abs):        {te_diff:.3f}")


In [8]:
# Cell 8: Baseline metrics
y_pred_baseline = model.predict(X_test)
compute_and_print_metrics(
    y_true=y_test.values,
    y_pred=y_pred_baseline,
    sensitive_feature=X_test['Gender'].values,
    privileged_value='Male',
    unprivileged_value='Female',
    label="Baseline"
)


Baseline Accuracy: 0.583

--- Baseline Fairness Metrics for Sensitive Attribute: Gender ---
Statistical Parity Difference (|unpriv - priv|): 0.650
Equalized Odds TPR Difference (abs):             0.713
Equalized Odds FPR Difference (abs):             0.578
Average Odds Difference (OAED, abs):             0.067
Treatment Equality Difference (TED, abs):        3.830


In [9]:
# Cell 9: Re-weighing mitigation
df_rw = df.copy()
df_rw['privileged'] = df_rw['Gender'] == 'Male'
df_rw['favorable']  = df_rw['Best Match'] == 1

N            = len(df_rw)
count_priv   = df_rw['privileged'].sum()
count_unpriv = N - count_priv
count_fav    = df_rw['favorable'].sum()
count_unfav  = N - count_fav

def _weight(row):
    if   row['privileged'] and row['favorable']:
        return (count_priv * count_fav) / N**2
    elif (not row['privileged']) and row['favorable']:
        return (count_unpriv * count_fav) / N**2
    elif row['privileged'] and (not row['favorable']):
        return (count_priv * count_unfav) / N**2
    else:
        return (count_unpriv * count_unfav) / N**2

df_rw['instance_weight'] = df_rw.apply(_weight, axis=1)

vectorizer_rw = CountVectorizer()
X_rw = vectorizer_rw.fit_transform(df_rw['Resume'])
y_rw = df_rw['Best Match']
w_rw = df_rw['instance_weight']

model_rw = MultinomialNB()
model_rw.fit(X_rw, y_rw, sample_weight=w_rw)

y_pred_rw = model_rw.predict(X_rw)
print("Re-weighing — Accuracy:", accuracy_score(y_rw, y_pred_rw))
compute_and_print_metrics(
    y_true=y_rw.values,
    y_pred=y_pred_rw,
    sensitive_feature=df_rw['Gender'].values,
    privileged_value='Male',
    unprivileged_value='Female',
    label="Re-weighing"
)


Re-weighing — Accuracy: 0.5618
Re-weighing Accuracy: 0.562

--- Re-weighing Fairness Metrics for Sensitive Attribute: Gender ---
Statistical Parity Difference (|unpriv - priv|): 0.019
Equalized Odds TPR Difference (abs):             0.089
Equalized Odds FPR Difference (abs):             0.112
Average Odds Difference (OAED, abs):             0.101
Treatment Equality Difference (TED, abs):        1.684


In [10]:
# Cell 10: Exponentiated Gradient mitigation (with dense conversion)
vectorizer_eg  = CountVectorizer()
X_sparse       = vectorizer_eg.fit_transform(df['Resume'])
y_full         = df['Best Match']
sensitive_full = df['Gender']

# split (still sparse)
X_train_eg, X_test_eg, \
y_train_eg, y_test_eg, \
sens_train, sens_test = train_test_split(
    X_sparse, y_full, sensitive_full,
    test_size=0.2,
    stratify=y_full,
    random_state=42
)

# convert to dense arrays
X_train_eg_arr = X_train_eg.toarray()
X_test_eg_arr  = X_test_eg.toarray()

mitigator = ExponentiatedGradient(
    estimator=MultinomialNB(),
    constraints=DemographicParity()
)
mitigator.fit(
    X_train_eg_arr,
    y_train_eg,
    sensitive_features=sens_train
)

y_pred_eg = mitigator.predict(X_test_eg_arr)
print("Exponentiated Gradient — Accuracy:", accuracy_score(y_test_eg, y_pred_eg))
compute_and_print_metrics(
    y_true=y_test_eg.values,
    y_pred=y_pred_eg,
    sensitive_feature=sens_test.values,
    privileged_value='Male',
    unprivileged_value='Female',
    label="Exponentiated Gradient"
)


Exponentiated Gradient — Accuracy: 0.51
Exponentiated Gradient Accuracy: 0.510

--- Exponentiated Gradient Fairness Metrics for Sensitive Attribute: Gender ---
Statistical Parity Difference (|unpriv - priv|): 0.004
Equalized Odds TPR Difference (abs):             0.041
Equalized Odds FPR Difference (abs):             0.040
Average Odds Difference (OAED, abs):             0.041
Treatment Equality Difference (TED, abs):        1.299


In [11]:
# Cell 11: Threshold Optimizer mitigation (with dense conversion)
postproc = ThresholdOptimizer(
    estimator=mitigator,
    constraints="demographic_parity",
    predict_method="predict",
    prefit=True
)
postproc.fit(
    X_train_eg_arr,
    y_train_eg,
    sensitive_features=sens_train
)

y_pred_to = postproc.predict(
    X_test_eg_arr,
    sensitive_features=sens_test
)
print("Threshold Optimizer — Accuracy:", accuracy_score(y_test_eg, y_pred_to))
compute_and_print_metrics(
    y_true=y_test_eg.values,
    y_pred=y_pred_to,
    sensitive_feature=sens_test.values,
    privileged_value='Male',
    unprivileged_value='Female',
    label="Threshold Optimizer"
)


Threshold Optimizer — Accuracy: 0.508
Threshold Optimizer Accuracy: 0.508

--- Threshold Optimizer Fairness Metrics for Sensitive Attribute: Gender ---
Statistical Parity Difference (|unpriv - priv|): 0.009
Equalized Odds TPR Difference (abs):             0.026
Equalized Odds FPR Difference (abs):             0.052
Average Odds Difference (OAED, abs):             0.039
Treatment Equality Difference (TED, abs):        1.361
